In [1]:
#____importing Libraries_____#
import sklearn.neighbors
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer

In [2]:

#____Setting up HyperParameters____#
cats = ['rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.space', 'rec.motorcycles', 'misc.forsale']
newsgroups = fetch_20newsgroups(subset='all')
vectorizer = TfidfVectorizer()
# corpus = ['This is the first document.','This document is the second document.','And this is the third one.','Is this the first document?']
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# print(vectorizer.get_feature_names())


#___ Preprocessing on data using Tf-Idf_____#
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target
# print(vectorizer.get_feature_names())
print(X.shape)
# print(y)
weights = 'uniform' # Using Unifor Disturbuted for weight initialization

#___ defining the classifier(KNN)____#
clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=1, weights=weights)

#___ K-Fold (5-Fold) ___#


(18846, 173762)


In [3]:
def cross_validation(data, target, classifier, cv=5):
    return sklearn.model_selection.cross_val_score(classifier, data, target, cv=cv)

def test_classifier(X, y, clf, test_size=0.4, y_names=None):
    # train-test split
    print('test size is: %2.0f%%' % (test_size * 100))
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=test_size)
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    print ("accuracy before cross-validation is %0.2f" % accuracy_score(y_test, y_predicted))
    scores = cross_validation(X, y, clf, cv=5)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print('Classification report:')
    print(sklearn.metrics.classification_report(y_test, y_predicted, target_names=y_names))
test_classifier(X,y, clf, test_size=0.2, y_names=None)

test size is: 20%
accuracy before cross-validation is 0.84
Accuracy: 0.83 (+/- 0.01)
Classification report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       167
           1       0.80      0.70      0.75       195
           2       0.74      0.76      0.75       180
           3       0.77      0.70      0.73       204
           4       0.76      0.82      0.79       195
           5       0.89      0.80      0.85       204
           6       0.76      0.65      0.70       211
           7       0.88      0.83      0.85       204
           8       0.93      0.92      0.93       202
           9       0.88      0.93      0.90       190
          10       0.91      0.93      0.92       206
          11       0.91      0.91      0.91       211
          12       0.77      0.82      0.80       165
          13       0.90      0.89      0.89       212
          14       0.88      0.88      0.88       184
          15       0.90    

In [4]:
#_______Import libraries_______#
import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import check_classification_targets
# from sklearn.datasets import fetch_20newsgroups

#______Define hyper parameters______#
cats = ['rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.space', 'rec.motorcycles', 'misc.forsale']
metric = 'euclidean'
#_____Load data_____#
newsgroups = fetch_20newsgroups(subset='train', categories=cats, remove=('headers', 'footers', 'quotes'))
#_____Preprocessing_____#
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data) # Convert to Tf-Idf vectorization
y = newsgroups.target



check_classification_targets(y) # Ensure that target y is of a non-regression type.


n_samples, n_features = X.shape
#_____Defining classes(centroids)______#
le = LabelEncoder()
y_indices = le.fit_transform(y)
classes = le.classes_
n_classes = classes.size

# Mask mapping each class to its members.
centroids = np.empty((n_classes, n_features), dtype=np.float64)
# Number of clusters in each class.
n_cluster = np.zeros(n_classes)

for current_class in range(n_classes):
    center_mask = y_indices == current_class
    n_cluster[current_class] = np.sum(center_mask)
    centroids[current_class] = X[center_mask].mean(axis=0)

In [9]:
def get_vectorizer_array(query):
    return vectorizer.transform([query]).toarray()


def pred(X):
    return classes[pairwise_distances(X, centroids, metric=metric).argmin(axis=1)]


	
#____Get test Data_____#
newsgroups_test = fetch_20newsgroups(subset='test', categories=cats, remove=('headers', 'footers', 'quotes'))
x_testdata = newsgroups_test.data
y_test = newsgroups_test.target
testdata = [[a_, b_] for a_, b_ in zip(x_testdata, y_test)]

correct = sum(str(pred(get_vectorizer_array(testcase[0]))[0]) == str(testcase[1]) for testcase in testdata)


# Print the accurency in percentage
result = str(correct / len(testdata))
result

'0.7080168776371308'